In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Get MNIST dataset
def get_mnist():
    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    
    x_train, y_train = mnist[0].next_batch(55000)
    x_valid, y_valid = mnist[1].next_batch(5000)
    x_test, y_test = mnist[2].next_batch(10000)
        
    return ([x_train, y_train], [x_valid, y_valid], [x_test, y_test])

In [3]:
def display_image(x, y, ix):
    x = x[ix]
    img = np.reshape(x, (28, 28))
    y = y[ix]

    plt.imshow(1-img, cmap=plt.get_cmap('gray'))
    plt.show()
    print np.where(y != 0)[0][0]

In [20]:
# Neural network class
class neural_net:
    def __init__(self, layers=[784, 10], activations=[tf.nn.softmax], error=tf.contrib.losses.log_loss):
        self.n_layers = len(layers)
        self.n_inputs = layers[0]
        self.n_outputs = layers[-1]
        
        self.x = tf.placeholder('float', [None, self.n_inputs], name='x')
        self.y = tf.placeholder('float', [None, self.n_outputs], name='y')
        
        self.W = []
        self.B = []
        self.neural_layers = [self.x]
        self.trained_W = None
        self.trained_B = None
                
        for layer in xrange(self.n_layers-1):
            input_layer = self.neural_layers[-1]
            
            w = tf.Variable(tf.random_uniform([layers[layer], layers[layer+1]], name='rng_W_'+str(layer)), 
                            name='W_'+str(layer))
            b = tf.Variable(tf.random_uniform([layers[layer+1]], name='rng_B_'+str(layer)), name='B_'+str(layer))
            
            output_layer = tf.add(tf.matmul(input_layer, w, name='W_'+str(layer)+'times_x'), 
                                  b, name='W_'+str(layer)+'_times_x_plus_B_'+str(layer))
            self.neural_layers += [activations[layer](output_layer)]
            
            self.W += [w]
            self.B += [b]
            
        self.error = error(self.neural_layers[-1], self.y)
        
    def train(self, train_x, train_y, step_size=0.01, batch_size=100, max_iters=1):
        train_step = tf.train.AdamOptimizer(step_size, name='grad_desc').minimize(self.error)
        
        train_size, n_features = train_x.shape
        train_x = np.vsplit(train_x, train_size/batch_size)
        train_y = np.vsplit(train_y, train_size/batch_size)
        
        with tf.Session() as session:
            tf.train.SummaryWriter('mlp_logs', session.graph)
            session.run(tf.initialize_all_variables())
            
            for i in xrange(max_iters):
                for batch_x, batch_y in zip(train_x, train_y):
                    session.run(train_step, feed_dict={self.x: batch_x, self.y: batch_y})
                    
            W = session.run(self.W)
            B = session.run(self.B)
        
        self.trained_W = W
        self.trained_B = B
        return (W, B)
        
    def predict(self, test_x):
        with tf.Session() as session:
            for i in xrange(self.n_layers-1):
                if self.trained_W == None and self.trained_B == None:
                    session.run(tf.initialize_all_variables())
                else:
                    session.run(self.W[i].assign(self.trained_W[i]))
                    session.run(self.B[i].assign(self.trained_B[i]))
            return session.run(self.neural_layers[-1], feed_dict={self.x: test_x})
        
    def unlearn(self):
        self.trained_W = None
        self.trained_B = None
        with tf.Session() as session:
            session.run(tf.initialize_all_variables)

In [21]:
[x_train, y_train], [x_valid, y_valid], [x_test, y_test] = get_mnist()

def cross_entropy(predictions, labels):
    log_pred = tf.log(predictions)
    cross_entropy = -tf.mul(labels, log_pred)
    return tf.reduce_mean(cross_entropy)

def identity(x):
    return x

mlp = neural_net(layers=[784, 10], activations=[tf.nn.softmax], error=cross_entropy)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
W, B = mlp.train(x_train, y_train, max_iters=2)

In [23]:
y_pred = mlp.predict(x_train)

In [24]:
np.sum(np.argmax(y_pred, 1) != np.argmax(y_train, 1))

4939

In [25]:
y_pred = mlp.predict(x_test)
np.sum(np.argmax(y_pred, 1) != np.argmax(y_test, 1))

958